In [5]:
import os
import glob
import librosa
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Dropout, LSTM
from keras.layers import TimeDistributed 
import tkinter as tk
from tkinter import filedialog
from pydub import AudioSegment
import sounddevice as sd
from PIL import Image, ImageTk 

In [6]:
def extract_features(file_path):
    audio, sr = librosa.load(file_path, res_type='kaiser_fast')
    features = np.mean(librosa.feature.mfcc(y=audio, sr=sr, n_mfcc=13).T, axis=0)
    return features

data = []
labels = []
directory_path = r'C:\Users\S M N RAZA\Downloads\archive\TESS Toronto emotional speech set data\TESS Toronto emotional speech set data'

for folder_name in os.listdir(directory_path):
    folder_path = os.path.join(directory_path, folder_name)
    for file_path in glob.glob(os.path.join(folder_path, '*.wav')):
        print(file_path)
        features = extract_features(file_path)
        data.append(features)
        labels.append(folder_name)

C:\Users\S M N RAZA\Downloads\archive\TESS Toronto emotional speech set data\TESS Toronto emotional speech set data\OAF_angry\OAF_back_angry.wav
C:\Users\S M N RAZA\Downloads\archive\TESS Toronto emotional speech set data\TESS Toronto emotional speech set data\OAF_angry\OAF_bar_angry.wav
C:\Users\S M N RAZA\Downloads\archive\TESS Toronto emotional speech set data\TESS Toronto emotional speech set data\OAF_angry\OAF_base_angry.wav
C:\Users\S M N RAZA\Downloads\archive\TESS Toronto emotional speech set data\TESS Toronto emotional speech set data\OAF_angry\OAF_bath_angry.wav
C:\Users\S M N RAZA\Downloads\archive\TESS Toronto emotional speech set data\TESS Toronto emotional speech set data\OAF_angry\OAF_bean_angry.wav
C:\Users\S M N RAZA\Downloads\archive\TESS Toronto emotional speech set data\TESS Toronto emotional speech set data\OAF_angry\OAF_beg_angry.wav
C:\Users\S M N RAZA\Downloads\archive\TESS Toronto emotional speech set data\TESS Toronto emotional speech set data\OAF_angry\OAF_bi

KeyboardInterrupt: 

In [ ]:


def predict_emotion(audio_file):
    features = extract_features(audio_file)
    features = features[np.newaxis, np.newaxis, :]  
    features = features.reshape(features.shape[0], -1)

    print("Features shape:", features.shape)
    print("Features:", features)

    predicted_probabilities = model.predict(features)
    print("Predicted probabilities shape:", predicted_probabilities.shape)
    print("Predicted probabilities:", predicted_probabilities)

    predicted_label_index = np.argmax(predicted_probabilities)
    print("Predicted label index:", predicted_label_index)

    predicted_emotion = label_encoder.classes_[predicted_label_index]
    print("Predicted emotion:", predicted_emotion)


    # Emotion mapping for TESS dataset
    emotion_mapping = {
        'YAF_angry': 'ANGRY',
        'YAF_disgust': 'DISGUST',
        'YAF_fear': 'FEAR',
        'YAF_happy': 'HAPPY',
        'YAF_neutral': 'NEUTRAL',
        'YAF_pleasant_surprised': 'SURPRISED',
        'YAF_sad': 'SAD',
        'OAF_angry': 'ANGRY',
        'OAF_disgust': 'DISGUST',
        'OAF_Fear': 'FEAR',
        'OAF_happy': 'HAPPY',
        'OAF_neutral': 'NEUTRAL',
        'OAF_Pleasant_surprised': 'SURPRISED',
        'OAF_Sad': 'SAD',
    }


    recognizable_emotion = emotion_mapping.get(predicted_emotion)
    return recognizable_emotion


In [ ]:

class EmotionApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Emotion Prediction App")
        self.root.configure(bg='yellow')
        
        self.emotion_to_emoji = {
            "HAPPY": "happy.png",
            "SAD": "sad.png",
            "ANGRY": "angry.png",
            "SURPRISED": "surprised.png",
            "NEUTRAL": "neutral.png",
            "FEAR": "fear.png",
            "DISGUST": "disgust.png"
        }
        
        self.emoji_image = None  
        self.prediction_history = []  
        
        self.show_home_page()
        
    def show_home_page(self):
        self.clear_window()
        
        label = tk.Label(self.root, text="Welcome to Emotion Prediction App", font=('Helvetica bold', 16))
        label.pack(pady=20)
        
        button = tk.Button(self.root, text="Audio Prediction", command=self.show_audio_page, bg='orange')
        button.pack()
        
        button_history = tk.Button(self.root, text="Prediction History", command=self.show_history_page, bg='lightgreen')
        button_history.pack(pady=10)
        
        about_button = tk.Button(self.root, text="About The App", command=self.show_about_page, bg='lightblue')
        about_button.pack(pady=10)
        
    def show_audio_page(self):
        self.clear_window()
        
        canvas = tk.Canvas(self.root, width=500, height=500, bg='green')
        canvas.pack()
        
        label1 = tk.Label(self.root, text='SPEECH EMOTION', font=('Helvetica bold', 26))
        canvas.create_window(250, 50, window=label1)
        
        def upload_audio():
            file_path = filedialog.askopenfilename(filetypes=[("Audio Files", "*.wav")])
            if file_path:
                predicted_emotion = predict_emotion(file_path)
                label2.config(text=predicted_emotion)
                
                self.prediction_history.append((os.path.basename(file_path), predicted_emotion))
                
                emoji_image_path = r'C:\Users\S M N RAZA\Downloads\Emotion Emojis\Emotion Emojis'
                if emoji_image_path:
                    emoji_image = Image.open(emoji_image_path)
                    emoji_image = emoji_image.resize((100, 100), Image.ANTIALIAS)
                    self.emoji_image = ImageTk.PhotoImage(emoji_image)
                    emoji_label.config(image=self.emoji_image)
                
        button1 = tk.Button(self.root, text='Upload Audio', command=upload_audio, bg='orange')
        canvas.create_window(250, 150, window=button1)
        
        label2 = tk.Label(self.root, text='Predicted Emotion Will Be Displayed Here')
        canvas.create_window(250, 200, window=label2)
        
        emoji_label = tk.Label(self.root, image=None)
        canvas.create_window(250, 300, window=emoji_label)
        
        back_button = tk.Button(self.root, text="Back to Home", command=self.show_home_page)
        canvas.create_window(250, 400, window=back_button)
        
    def show_history_page(self):
        self.clear_window()
        
        canvas = tk.Canvas(self.root, width=500, height=500, bg='lightgreen')
        canvas.pack()
        
        label = tk.Label(self.root, text="Prediction History", font=('Helvetica bold', 16))
        canvas.create_window(250, 50, window=label)
        
        if self.prediction_history:
            for index, (file_name, predicted_emotion) in enumerate(self.prediction_history, start=1):
                history_text = f"{index}. File: {file_name}, Emotion: {predicted_emotion}"
                history_label = tk.Label(self.root, text=history_text)
                canvas.create_window(250, 100 + index * 30, window=history_label)
        else:
            no_history_label = tk.Label(self.root, text="No prediction history available.")
            canvas.create_window(250, 150, window=no_history_label)
        
        back_button = tk.Button(self.root, text="Back to Home", command=self.show_home_page)
        canvas.create_window(250, 450, window=back_button)
        
    def show_about_page(self):
        self.clear_window()
        
        canvas = tk.Canvas(self.root, width=500, height=500, bg='skyblue')
        canvas.pack()
        
        label = tk.Label(self.root, text="About The Software", font=('Helvetica bold', 16))
        canvas.create_window(250, 50, window=label)
        
        about_text = ("Hello Everyone !! "
                      " Speech Emotion Recognition is a software that recognizes the emotion of the user."
                      " All of the audio files in this software should be inputted with '.wav' extension."
                      " A special thanks to the University of Toronto for the TESS data set and to Manjushree Mam "
                      "  that guided me throughout the journey of making this software.")
        
        about_label = tk.Label(self.root, text=about_text, wraplength=400)
        canvas.create_window(250, 150, window=about_label)
        
        back_button = tk.Button(self.root, text="Back to Home", command=self.show_home_page)
        canvas.create_window(250, 400, window=back_button)
        
    def clear_window(self):
        for widget in self.root.winfo_children():
            widget.destroy()

if __name__ == "__main__":
    root = tk.Tk()
    app = EmotionApp(root)
    root.mainloop()

Features shape: (1, 13)
Features: [[-3.7500354e+02  1.3827100e+02  1.9743643e+01  3.0104395e+01
   9.0152121e+00  1.2175241e+01 -5.4135976e+00  7.0361738e+00
  -2.5249922e+00 -5.0353045e+00  3.1406000e-01 -1.1545728e+00
  -1.8910319e+00]]


Exception in Tkinter callback
Traceback (most recent call last):
  File "c:\Users\S M N RAZA\AppData\Local\Programs\Python\Python311\Lib\tkinter\__init__.py", line 1948, in __call__
    return self.func(*args)
           ^^^^^^^^^^^^^^^^
  File "C:\Users\S M N RAZA\AppData\Local\Temp\ipykernel_21460\3001106129.py", line 49, in upload_audio
    predicted_emotion = predict_emotion(file_path)
                        ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\S M N RAZA\AppData\Local\Temp\ipykernel_21460\2699127468.py", line 9, in predict_emotion
    predicted_probabilities = model.predict(features)
                              ^^^^^
NameError: name 'model' is not defined
